# Create custom federated architectures using FLEXible

In this notebook we show how to create customs architetures when working FLEXible. We hope that this notebook will help users to learn how to use the `FlexPool` class in case they want to create different architectures from the two available in the `FlexPool` class. Those methods are, `client_server_architecture` and `p2p_architecture`, and are the most probable type of architectures that might be found during a federated learning experiment, but it might be interesting to create another architecture.

In [ ]:
from flex.pool import FlexPool
from flex.actors import FlexActors, FlexRole
from flex.data import FedDataDistribution, Dataset

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In this example we are going to use a simple dataset. This dataset is the Iris dataset, available in the sklearn library. Then, we are going to federate it as we need to have a Federated Dataset to create customs architectures using `FlexPool`.

In [ ]:
iris = load_iris()
# Generate train-test splits
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.33, random_state=42)

train_iris = Dataset(X_train, y_train)
test_iris = Dataset(X_test, y_test)

federated_iris = FedDataDistribution.iid_distribution(train_iris, n_clients=5)

Now it's time to create some functions to create custom architectures.

In [ ]:
def custom_architecture_clients_as_aggregators(clients_ids: list) -> FlexActors :
    if "server" in clients_ids:
        raise ValueError(
            "The name 'server' is reserved only for the server in a client-server architecture."
        )

    actors = FlexActors(
        {client_id: FlexRole.aggregator_client for client_id in clients_ids}
    )

    actors['server'] = FlexRole.server

    return actors

In [ ]:
clients_aggregators_actors = custom_architecture_clients_as_aggregators(list(federated_iris.keys()))

clients_aggregators_pool = FlexPool(federated_iris, clients_aggregators_actors)

clients = clients_aggregators_pool.clients
aggregators = clients_aggregators_pool.aggregators
servers = clients_aggregators_pool.servers

print(f"Clients ids: {clients.actor_ids}")
print(f"Aggregators ids: {aggregators.actor_ids}")
print(f"Servers ids: {servers.actor_ids}")

print(f"Clients and aggregators are the same actors: {sorted(clients.actor_ids) == sorted(aggregators.actor_ids)}")

Now lets create an architecture where a number of clients will act as aggregators too.

In [ ]:
def custom_architecture_n_clients_as_aggregators(clients_ids: list, n_clients_as_aggregators) -> FlexActors :
    if "server" in clients_ids:
        raise ValueError(
            "The name 'server' is reserved only for the server in a client-server architecture."
        )

    n_clients_as_aggregators = min(n_clients_as_aggregators, len(clients_ids))

    actors = FlexActors(
        {clients_ids[client_id]: FlexRole.aggregator_client for client_id in range(n_clients_as_aggregators)}
    )

    actors.update(
        FlexActors({clients_ids[client_id]: FlexRole.client for client_id in range(n_clients_as_aggregators, len(clients_ids))})
    )

    actors['server'] = FlexRole.server

    return actors

In [ ]:
n_clients_as_aggregators_actors = custom_architecture_n_clients_as_aggregators(list(federated_iris.keys()), n_clients_as_aggregators=5)

n_clients_as_aggregators_pool = FlexPool(federated_iris, n_clients_as_aggregators_actors)

clients = n_clients_as_aggregators_pool.clients
aggregators = n_clients_as_aggregators_pool.aggregators
servers = n_clients_as_aggregators_pool.servers

print(f"Clients ids: {clients.actor_ids}")
print(f"Aggregators ids: {aggregators.actor_ids}")
print(f"Servers ids: {servers.actor_ids}")

Now lets create a "random" custom architecture, that will be moreover manually created.

In [ ]:
def custom_architecture(clients_ids: list) -> FlexActors :
    if "server" in clients_ids:
        raise ValueError(
            "The name 'server' is reserved only for the server in a client-server architecture."
        )

    import numpy as np

    n_clients = len(clients_ids)
    print(f"Total number of clients: {n_clients}")
    n_clients_aggregators = np.random.randint(low=0, high=n_clients//2)
    print(f"Clients as aggregators: {n_clients_aggregators}")
    n_clients_servers = np.random.randint(low=0, high=n_clients-n_clients_aggregators)
    print(f"Clients as servers: {n_clients_servers}")
    rest_clients = n_clients - (n_clients_aggregators + n_clients_servers)
    print(f"Only clients: {rest_clients}")
    actors = FlexActors(
        {clients_ids[client_id]: FlexRole.aggregator_client for client_id in range(n_clients_aggregators)}
    )

    actors.update(
        FlexActors({clients_ids[client_id]: FlexRole.server_client for client_id in range(n_clients_aggregators, n_clients-rest_clients)})
    )

    actors.update(
        FlexActors({clients_ids[client_id]: FlexRole.client for client_id in range(n_clients_aggregators+n_clients_servers, n_clients)})
    )

    if n_clients_servers == 0:
        actors['server'] = FlexRole.server

    return actors

In [ ]:
custom_actors = custom_architecture(list(federated_iris.keys()))

custom_pool = FlexPool(federated_iris, custom_actors)

clients = custom_pool.clients
aggregators = custom_pool.aggregators
servers = custom_pool.servers

print(f"Clients ids: {clients.actor_ids}")
print(f"Aggregators ids: {aggregators.actor_ids}")
print(f"Servers ids: {servers.actor_ids}")

# END

We have shown how to create custom architetures using the FlexPool initializer instead of using the `class_methods` that are already implemented in the `FlexPool`class.